In [2]:
import pandas as pd

filename="DailyComments.csv"
dataframe_comments = pd.read_csv(filename)
dataframe_comments

,Day of Week,comments
0,Monday,"Hello, how are you?"
1,Tuesday,Today is a good day!
2,Wednesday,It's my birthday so it's a really special day!
3,Thursday,Today is neither a good day or a bad day!
4,Friday,I'm having a bad day.
5,Saturday,There' s nothing special happening today.
6,Sunday,Today is a SUPER good day!


In [10]:
filename="stock_data.csv"
dataframe_stockdata = pd.read_csv(filename)
dataframe_stockdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 5790
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       5791 non-null   object
 1   Sentiment  5791 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.6+ KB


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dataframe_stockdata['Text'],dataframe_stockdata['Sentiment'],test_size=0.3,random_state=101)

In [14]:
x_train

3644              With some luck BT 22 is current target.
3338    Automatic cuts starts today, european crisis g...
4061    COX Crocs: Growing Brand At A Fair Price. It m...
1102                                     AEG Over 28.23  
5405    Investors are showing more appetite for the ju...
                              ...                        
599     ed Weekly Triangle on MH,...Net Profit  4,895....
5695    Gold Futures Rise To Hover Near Rs 39,800 Amid...
1361    Anybody else hearing about one of the large cr...
1547    Green Weekly Triangle on PBY,....Open Sell Sho...
4959     Weekly.Barely hanging onto double support are...
Name: Text, Length: 4053, dtype: object

In [19]:
# Convert each entry into a term frequency-inverse document frequency (tfidf) vector 
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the tf-idf feature matrix
tfidf = TfidfVectorizer()
tfidf_matrix_test = tfidf.fit_transform(dataframe_comments['comments'].values)
tfidf_matrix = tfidf.fit_transform(x_train.values)
# show the matrix
# tfidf_matrix.toarray()

words = tfidf.get_feature_names()

tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(),columns=words)
# tfidf_matrix

,00,000,0025,00pm,01,02,028,03,04,05,...,œa,œessentialâ,œhysteresisâ,œi,œpandemic,œpriority,œrecalibratingâ,œsince,œwe,œyou
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
'''
Creating a custom Text normalization transformer
Text normalization reduces the number of dimensions,
decreasing sparsity. Besides the simple filtering of tokens (removing punctuation and
stopwords), there are two primary methods for text normalization: stemming and
lemmatization.

'''
import unicodedata
import nltk
from sklearn.base import BaseEstimator, TransformerMixin

class TextNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self, language='english'):
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = nltk.stem.WordNetLemmatizer()

    def is_punct(self, token):
        return all(
                    unicodedata.category(char).startswith('P') for char in token
                    )

    def is_stopword(self, token):
        return token.lower() in self.stopwords

    def lemmatize(self, token, pos_tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag[0], wn.NOUN)
        
        return self.lemmatizer.lemmatize(token, tag)

    def normalize(self, document):
        return [
            self.lemmatize(token, tag).lower()
            for paragraph in document
            for sentence in paragraph
            for (token, tag) in sentence
            if not self.is_punct(token) and not self.is_stopword(token)
        ]

    def fit(self, X, y=None):

        '''
        Finally, we must add the Transformer interface, allowing us to add this class to a
        Scikit-Learn pipeline, which we’ll explore in the next section:
        '''
        return self
    
    def transform(self, documents):
         for document in documents:
            yield self.normalize(document)


In [6]:
'''
Creating a custom Gensim vectorization transformer
Gensim vectorization techniques are an interesting case study because Gensim cor‐
pora can be saved and loaded from disk in such a way as to remain decoupled from
the pipeline.
'''
import os
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

from gensim.corpora import Dictionary
from gensim.matutils import sparse2full

class GensimVectorizer(BaseEstimator, TransformerMixin):

    def __init__(self, path=None):
        self.path = path
        self.id2word = None
        self.load()
    
    def load(self):
        if os.path.exists(self.path):
            self.id2word = Dictionary.load(self.path)
    
    def save(self):
        self.id2word.save(self.path)
    
    def fit(self, documents, labels=None):
        self.id2word = Dictionary(documents)
        self.save()
        return self

    def transform(self, documents):
        for document in documents:
            docvec = self.id2word.doc2bow(document)
            yield sparse2full(docvec, len(self.id2word))

In [20]:
'''
Scikit-Learn's Pipeline objects enable us to integrate a series of transformers that combine normal‐
ization, vectorization, and feature analysis into a single, well-defined mechanism.The purpose of a Pipeline is to chain together multiple estimators representing a fixed sequence of steps into a single unit. All estimators in the pipeline, except the last one, must be transformers. Pipelines are constructed by describing a list of (key, value) pairs where the key is a string that names the step and the value is the estimator object.
'''
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

model = Pipeline([
    ('normalizer', TextNormalizer()),
    ('vectorizer', GensimVectorizer),
    ('bayes', MultinomialNB()),
])
a = y_train
# model.named_steps['bayes']
# model.steps[2]
# model(tfidf_matrix)
# model.named_steps['bayes'].fit(tfidf_matrix,a).predict(tfidf_matrix)
model.named_steps['bayes'].fit(tfidf_matrix,a)
model.named_steps['bayes'].predict()

# model.named_steps['normalizer'].transform(tfidf_matrix)

MultinomialNB()

In [8]:
tfidf_matrix

,are,bad,birthday,day,good,happening,having,hello,how,is,...,neither,nothing,or,really,so,special,super,there,today,you
0,0.5,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.5,0.5,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.5
1,0.0,0.000000,0.0000,0.416629,0.547817,0.000000,0.000000,0.0,0.0,0.547817,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.475619,0.0
2,0.0,0.000000,0.3337,0.180070,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.3337,0.3337,0.277000,0.000000,0.000000,0.000000,0.0
3,0.0,0.362620,0.0000,0.471459,0.309956,0.000000,0.000000,0.0,0.0,0.309956,...,0.436847,0.000000,0.436847,0.0000,0.0000,0.000000,0.000000,0.000000,0.269106,0.0
4,0.0,0.589882,0.0000,0.383466,0.000000,0.000000,0.710628,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0
5,0.0,0.000000,0.0000,0.000000,0.000000,0.495772,0.000000,0.0,0.0,0.000000,...,0.000000,0.495772,0.000000,0.0000,0.0000,0.411533,0.000000,0.495772,0.305405,0.0
6,0.0,0.000000,0.0000,0.329775,0.433614,0.000000,0.000000,0.0,0.0,0.433614,...,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.611129,0.000000,0.376467,0.0
